# Agentes Arquitetos de sistemas

### Criar agentes que tenham a persona de arquiteto e que sejam capaz de propor soluções para construção de sistemas para uma empresa de telecomunicação

MVP do conceito de Multi-agent com o framework CrewAI

In [46]:
# Instalar dependencias para o framework, caso não estejam instaladas remover # da linha abaixo

# !pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [47]:
# Controle de alarmes
import warnings
warnings.filterwarnings('ignore')

In [48]:
# Importar classes do framework

from crewai import Agent, Task, Crew

- Os agente irão utilizar o LLM `gpt-3.5-turbo` da OpenAI, chamando a API direto.


In [49]:
import os

openai_api_key = os.getenv("OPENAI_API_KEY") # Chave de API da OpenAI definida nas variáveis de ambiente
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

## Criando Agentes

- Será Definido nos Agentes `goal` (papel), `goal` (objetivo/meta) e `backstory` (história de fundo).
- Foi observado que os LLMs (Modelos de Linguagem de Grande Escala) têm um desempenho melhor quando estão interpretando um papel, entrando de fato em uma persona

### Agent: Arquiteto corporativo

In [50]:
# Definição do agente Arquiteto corporativo

Arquiteto_corporativo = Agent(
    role="Arquiteto corporativo",
    goal="Arquitetar uma solução para desenvolvimento de um sistema para {topic}",
    backstory= ("Você está trabalhando como arquiteto corporativo em uma empresa de tecnologia."
              "especialiste em desenvolver sistemas para: {topic}."
              "Você tem conhecimento técnico e habilidades de gestão que integram tecnologia e negócio"
              "seu papel permeia estrategicamente a organização como um todo"
              "Você define métodos e ferramentas que irão atender toda a empresa"
              "tome decisao acerca de padronização de linguagens e frameworks a serem utilizados nas soluções, padronização de bancos de dados etc"
              "processos de controle de mudança para auditoria depublicações em ambiente produtivo"),
    allow_delegation=False,
	verbose=True
)

### Agent: Arquiteto de soluções

In [51]:
# Definição do agente Arquiteto de soluções

Arquiteto_de_solucoes = Agent(
    role="Utilizar os padrões de arquitetura corporativa para desenvolver uma solução para {topic}",
    goal="transformar o requisito de negócio em componentes de engenharia "
         "garante que a solução de software resolve um problema de negócio para {topic}"
         "gestão dos requisitos",
    backstory="criar e liderar processos de integração para construir a solução para o sistema de {topic}"
              "avaliar restrições de projeto determinadas pelo arquiteto corporativo"
              "analisar impactos comerciais de escolhas técnicas"
              "estar alinhado a estratégia técnica da organização"
              "trabalhar requisitos não funcionais e funcionais",
    allow_delegation=False,
    verbose=True
)

### Agent: Arquiteto de software

In [52]:
# Definição do agente Arquiteto de software

Arquiteto_de_software = Agent(
    role="Arquiteto de software",
    goal="Utilizar os padrões da solução definida pelo arquiteto de solução para desenvolver um sistema para {topic}.\n "
         "ser uma referência técnica para a equipe de desenvolvimento",
    backstory="atuação em momentos de crise.\n"
              "revisões de código para garantir os padrões arquiteturais,\n"
              "evangelizador das boas práticas e\n"
              "acompanhar desenvolvimento",
    allow_delegation=False,
    verbose=True
)

## Criando as Tasks

- Define your Tasks, and provide them a `description` (Descrição), `expected_output`(Retorno esperado) and `agent`(Qual agente é o responsavel).

### Task: Corporativa

In [53]:
Corporativa = Task(
    description=(
        "1. documentar modelo de arquitetura que deverá ser utilizada na"
            "construção do sistema de {topic}, ou seja definir a arquitetura corporativa que deve ser usada.\n"
            "Definir opções tecnologias, linguagens de programação, frameworks, bancos de dados, etc.\n"
        "2. avaliar custos das soluções propostas para implementar"
            "\n"
        "3. Desenvolver uma solução corporativa de DevOps "
            "\n"
        "4. avaliar os custos de opex da soluções propostas para implementar"
    ),
    expected_output="Um documento contendo a arquitetura corporativa a ser utilizada na construção do sistema de {topic}"
        "custo estimado para implementação (capex) da solução"
        "custo estimado para operação (opex) da solução para os próximos 10 anos"
        "Criar um desenho da solução a ser implementada, com todos os modulos a ser criado no sistema",
    agent=Arquiteto_corporativo,
)

### Task: Soluções

In [54]:
solução = Task(
    description=(
        "1. documentar um fluxo arquitetal utilizando a arquitetura corporativa definida pelo arquiteto corporativo"
            "utilizando as definições do arquiteto corporativo para construir um sistema para {topic}.\n"
        "2.  Documentar as tecnologias, linguagens e frameworks devem ser usado"
            "para construir o sistema para {topic}.\n"
		"3. avaliar restrições de projeto determinadas pelo arquiteto corporativo\n"
    ),
    expected_output="Um documento contendo a solução e desenho a utilizada na construção do sistema de {topic}"
    "Criar um desenho da solução a ser implementada, com todos os modulos a ser criado no sistema",
    agent=Arquiteto_de_solucoes,
)

### Task: Software

In [55]:
software = Task(
    description=(
        "1.documentar a arquitetura de software a ser usada utilizando a solução definida pelo arquiteto de solução.\n"
        "2. Definir os padrões de codificação a serem utilizados na construção do sistema para {topic}.\n"
        "3. Definir os padrões de testes a serem utilizados na construção do sistema para {topic}.\n"
        "4. Definir os padrões de documentação a serem utilizados na construção do sistema para {topic}.\n"),
    expected_output="Documento contendo a arquitetura de software a ser utilizada na construção do sistema de {topic}."
        "Documentar Padrões de codificação, testes e documentação a serem utilizados na construção do sistema de {topic}."
        "Revisões de código para garantir os padrões arquiteturais"
        "Criar um desenho da solução a ser implementada, com todos os modulos a ser criado no sistema",
    agent=Arquiteto_de_software
)

## Criando Crew

- `agents=` Utilizar as personas dos agentes criadas, arquiteto corp, soluções e software
- `tasks=`  Utilizar as tarefas criadas.
* Sobre `agents=` e `tasks=`, as tarefas serão executadas sequencialmente (ou seja, são dependentes umas das outras), portanto a ordem da tarefa na lista importa.
- `verbose=2` permite que você veja todos os logs da execução.

In [56]:
crew = Crew(
    agents=[Arquiteto_corporativo, Arquiteto_de_solucoes, Arquiteto_de_software],
    tasks=[Corporativa, solução, software],
    verbose=2
)

2024-08-14 14:58:41,771 - 140383099133952 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


## Inicando execução dos agentes CrewAI

In [57]:
# Execução dos agentes

topic = "Calcular o score de crédito dos clientes que precisam de internet fixa" # Aqui define o tópico que os agentes precisam trabalhar!
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Arquiteto corporativo
 [INFO]: == Starting Task: 1. documentar modelo de arquitetura que deverá ser utilizada naconstrução do sistema de Calcular o score de crédito dos clientes que precisam de internet fixa, ou seja definir a arquitetura corporativa que deve ser usada.
Definir opções tecnologias, linguagens de programação, frameworks, bancos de dados, etc.
2. avaliar custos das soluções propostas para implementar
3. Desenvolver uma solução corporativa de DevOps 
4. avaliar os custos de opex da soluções propostas para implementar


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer
Final Answer: My best complete final answer to the task.

---

### Documento de Arquitetura Corporativa para o Sistema de Cálculo de Score de Crédito

#### 1. Modelo de Arquitetura

##### 1.1. Arquitetura Corporativa

Para a construção do sistema de cálculo de score de crédito dos clientes que precisam de internet fixa, será utilizada uma arquitetura 

### Tradução do retorno dos agentes

In [58]:
# Traduzir texto para o portugues brasil com uma chamada da OpenAI
import openai
import os

# Chave de API da OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")

# Recebendo retorno dos agentes
Return_agent = result

# Função para traduzir texto para o portugues brasil com OpenAI
def traduzir_texto(Return_agent):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {
            "role": "user",
            "content": f"Traduzir texto, para o portugues brasil, segue texto:\n\n {Return_agent}"
        }])
    response = response.choices[0].message.content
    return response

# Traduzindo texto
Return_agent_translated = traduzir_texto(Return_agent) 

In [59]:
# Exibir resultado final
from IPython.display import Markdown
Markdown(Return_agent_translated)

Minha melhor resposta final completa para a tarefa.

---

**Documento de Arquitetura para o Sistema de Cálculo de Score de Crédito dos Clientes que Precisam de Internet Fixa**

---

### 1. Fluxo Arquitetural

#### 1.1. Visão Geral
A arquitetura do sistema será baseada em uma abordagem de microserviços para garantir escalabilidade, flexibilidade e manutenção eficiente. Cada componente será responsável por uma funcionalidade específica, comunicando-se através de APIs RESTful.

#### 1.2. Componentes Principais
- **Frontend**: Interface do usuário desenvolvida em React.js para interações diretas com os clientes.
- **Backend**: Microserviços desenvolvidos em Node.js para lógica de negócios e processamento de dados.
- **Banco de Dados**: MongoDB para armazenamento de dados não estruturados e PostgreSQL para dados estruturados.
- **Autenticação e Autorização**: Utilização do OAuth 2.0 para segurança e controle de acesso.
- **Integração de Dados**: Apache Kafka para streaming de dados e integração com serviços externos (e.g., bureaus de crédito).
- **Monitoramento e Logging**: ELK Stack (Elasticsearch, Logstash, Kibana) para monitoramento e análise de logs.
- **Orquestração de Contêineres**: Kubernetes para gerenciamento e orquestração de contêineres Docker.

### 2. Tecnologias, Linguagens e Frameworks

#### 2.1. Frontend
- **Linguagem**: JavaScript
- **Framework**: React.js
- **Bibliotecas Adicionais**: Redux para gerenciamento de estado, Axios para requisições HTTP

#### 2.2. Backend
- **Linguagem**: JavaScript (Node.js)
- **Framework**: Express.js
- **Bibliotecas Adicionais**: Mongoose para ODM com MongoDB, Sequelize para ORM com PostgreSQL

#### 2.3. Banco de Dados
- **NoSQL**: MongoDB
- **SQL**: PostgreSQL

#### 2.4. Integração de Dados
- **Streaming**: Apache Kafka

#### 2.5. Monitoramento e Logging
- **ELK Stack**: Elasticsearch, Logstash, Kibana

#### 2.6. Orquestração de Contêineres
- **Plataforma**: Kubernetes
- **Contêineres**: Docker

### 3. Avaliação das Restrições de Projeto

#### 3.1. Restrições de Segurança
- Implementação de OAuth 2.0 para autenticação e autorização de usuários.
- Criptografia de dados sensíveis em trânsito e em repouso usando TLS e AES-256.

#### 3.2. Restrições de Desempenho
- Garantir tempo de resposta inferior a 200ms para chamadas de APIs.
- Escalabilidade horizontal dos serviços utilizando Kubernetes.

#### 3.3. Restrições de Custo
- Utilização de serviços em nuvem como AWS ou Azure para reduzir CAPEX inicial.
- Monitoramento contínuo de OPEX, com estimativas de custos operacionais para os próximos 10 anos, incluindo dimensionamento automático para otimizar custos de infraestrutura.

### 4. Desenho da Solução

#### 4.1. Diagrama de Componentes
![Diagrama de Componentes](link-para-diagrama.png)

#### 4.2. Módulos a Serem Criados
- **Módulo de Interface do Usuário**: Desenvolvido em React.js, responsável por capturar dados do cliente e exibir o score de crédito.
- **Módulo de Processamento de Dados**: Microserviço em Node.js responsável por calcular o score de crédito com base nos dados fornecidos.
- **Módulo de Armazenamento de Dados**: Integração com MongoDB e PostgreSQL para armazenamento dos dados dos clientes e históricos.
- **Módulo de Integração Externa**: Microserviço em Node.js para comunicação com bureaus de crédito e outras fontes externas.
- **Módulo de Autenticação e Autorização**: Implementação de OAuth 2.0 para gerenciamento de acesso seguro.
- **Módulo de Monitoramento e Logging**: Configuração do ELK Stack para monitoramento contínuo e análise de logs.

### 5. Padrões de Codificação

#### 5.1. Convenções de Codificação
- **JavaScript**: Seguir o padrão de estilo Airbnb para JavaScript.
- **React.js**: Componentes funcionais com hooks.
- **Node.js**: Utilização de Promises e async/await para operações assíncronas.
- **MongoDB**: Utilização de Mongoose para definição de esquemas e validação de dados.
- **PostgreSQL**: Utilização de Sequelize ORM para definição de modelos e migrações.

#### 5.2. Revisão de Código
- Revisões de código obrigatórias para todas as pull requests.
- Utilização de ferramentas de análise estática de código como ESLint e Prettier.

### 6. Padrões de Testes

#### 6.1. Tipos de Testes
- **Testes Unitários**: Utilização de Jest para testes unitários em JavaScript.
- **Testes de Integração**: Utilização de Mocha e Chai para testes de integração.
- **Testes de Performance**: Utilização de Artillery para testes de carga e desempenho.
- **Testes de Segurança**: Utilização de OWASP ZAP para testes de vulnerabilidade.

#### 6.2. Cobertura de Testes
- Metas de cobertura de testes acima de 80% para todos os módulos.
- Relatórios de cobertura gerados automaticamente após cada build.

### 7. Padrões de Documentação

#### 7.1. Documentação de Código
- Comentários claros e concisos para todas as funções e classes.
- Utilização de JSDoc para geração automática de documentação de código.

#### 7.2. Documentação Técnica
- Criação de um Wiki no GitHub para centralizar toda a documentação técnica.
- Manuais de instalação, configuração e operação detalhados.
- Diagramas UML para ilustrar a arquitetura e os fluxos de dados.

#### 7.3. Documentação do Usuário
- Guias do usuário finais para clientes e operadores do sistema.
- FAQs e tutoriais passo-a-passo.

---

Espero que este documento atenda às expectativas e auxilie na construção de um sistema robusto e eficiente.